In [1]:
# Imports for data manipulation
import pandas as pd
import numpy as np

In [2]:
# Importa for data visualization
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Read sfdata (NO NEED TO RUN!)
sfdata_file = "../data/sf.csv"
sfdata = pd.read_csv(sfdata_file)

In [3]:
# Read bosdata
import re
import warnings

bos_file = '../data/boston.csv'
target_type = str  # The desired output type

with warnings.catch_warnings(record=True) as ws:
    warnings.simplefilter("always")

    bosData = pd.read_csv(bos_file, sep=",", header=0)
    print("Warnings raised:", ws)
    # We have an error on specific columns, try and load them as string
    for w in ws:
        s = str(w.message)
        print("Warning message:", s)
        match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
        if match:
            columns = match.group(1).split(',') # Get columns as a list
            columns = [int(c) for c in columns]
            print("Applying %s dtype to columns:" % target_type, columns)
            bosData.iloc[:,columns] = bosData.iloc[:,columns].astype(target_type)

('Warnings raised:', [<warnings.WarningMessage object at 0x7f9659226550>])
('Warning message:', 'Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.')
("Applying <type 'str'> dtype to columns:", [10])


In [ ]:
# Add a crime-count column to quickly count the data
bosData['CrimeCount'] = 1
sfdata['CrimeCount'] = 1

In [ ]:
def plotCrimeCounts(data, features, city):
    '''
    Data is the dataframe containing the crime statistics.
    Features is  disctionary mapping {description: column}
    city is the name of the city for which we are plotting.
    '''
    for description, columnName in features.iteritems():
        sns.countplot(y=columnName, data=data)
        plt.title('Crimes in {} by {}'.format(city, description))
        plt.show()

In [ ]:
# Create histograms on some important metrics
# Map from english_description : column name
columnsToPlotSF = { 'Police District' : 'PdDistrict',
                    'Crime Type' : 'Category',
                    'Day of the Week' : 'DayOfWeek',
                   'Crime Outcome' : 'Resolution' }

In [ ]:
plotCrimeCounts(sfdata, columnsToPlotSF, 'San Francisco')

In [ ]:
# Create histograms on some important metrics for Boston
columnsToPlotBos = { 'Reporting District' : 'REPTDISTRICT', 
                     'Weapon Type' : 'WEAPONTYPE',
                     'Shooting/No Shooting' : 'Shooting',
                     'Officer Shift' : 'SHIFT',
                     'Year' : 'Year', 
                     'Month' : 'Month',
                     'Day of the Week' : 'DAY_WEEK'}

In [ ]:
plotCrimeCounts(bosData, columnsToPlotBos, 'Boston')

In [ ]:
loadPickle = True

In [ ]:
# Convert date to actual date format. This might take a while!
# Note that we should not have to do this once the data
# has been cleaned!
# SKIP IF CLEAN DATA EXISTS
if not loadPickle:
    sfdata.Date = sfdata['Date'].apply(lambda x: pd.to_datetime(x, errors='raise'))
    sfdata.Time = sfdata['Time'].apply(lambda x: pd.to_datetime(x, errors='raise'))

In [4]:
# Helper functions to help clean SF data. We do this only once, as we 
# now save the results using pickle!
def buckets(series, n):
    # Takes a series and returns a series mapping each element to a
    # one of n buckets.
    mi, ma = series.min(), series.max()
    buckets = np.linspace(mi, ma, n)
    # print buckets
    def f(e):
        for i, el in enumerate(buckets):
            if e < el:
                return i
        return n - 1
            
    res = series.map(f)
    return res

def cleanColumns(data):
    # Used to rename the columns in our data grame to their appropriate names.
    # Also drops unnecessary columns.
    data['Latitude'] = data['Y']
    data['Longitude'] = data['X']
    data['Type'] = data['Category']
    
    # print data.columns
    data = data.drop(['IncidntNum', 'Descript','Resolution','Address','X','Y', 'Location'], axis=1)
    
    return data

def createPartitions(data, n):
    # Remove outliers from the latitude/longitude issues
    # We know that the city lies between -130, -120 longitude
    # We also know that the citiy lies between 37 and 40 degrees latitude
    data = data[-120 > data.Longitude][data.Longitude > (-130)]
    data = data[data.Latitude > 37][data.Latitude < 40]
    
    # Each row is an occurrance of a single crime. 
    # Keep around the original data
    data['Region'] =  n *  buckets(data['Latitude'], n) + buckets(data['Longitude'],n) + 1
    data['xRegion'] = buckets(data['Latitude'], n)
    data['yRegion'] =  buckets(data['Longitude'],n) + 1
    data['Region'] =  n * data.yRegion + data.xRegion    
    
    # Add in the types into the results.
    mapping = {key: i for i,key in enumerate(data['Type'].unique())}
    data['TypeIndex'] = data['Type'].map(mapping)

    # Now we can add the crime counts. 
    data['CrimeCount'] = np.ones(len(data))
    return data

def extractDateFeatures(data):
    # Creates a new data frame and returns it as copy with all the data that we're interested in
    # Create map from week days to integers
    DayOfWeek = {'Sunday': 1,
                 'Monday': 2,
                 'Tuesday': 3,
                 'Wednesday': 4,
                 'Thursday': 5,
                 'Friday': 6,
                 'Saturday': 7 }
    data['DoW'] = data['DayOfWeek'].map(DayOfWeek)
    data = data.drop('DayOfWeek', axis=1)
    print "Created Weeks"
    
    # We assume that the Date column is already in datetime format
    data['Month'] = data.Date.map(lambda x: x.month)
    data['DoM'] = data.Date.map(lambda x: x.day)
    data['Year'] = data.Date.map(lambda x: x.year) - data.Date.min().year
    data['ToD'] = data.Time.map(lambda x: x.minute)
    data['Time'] = data.Time.map(lambda x: x.value / 10**9) - data.Date.min().value / 10**9
    
    # We add an additional column that combines the month and the year into number of months since beginning
    data['TimeFeature'] = data.ix[:, ['Year', 'Month']].apply(lambda s: 12*s[0] + s[1], axis=1)
    
    data = data.drop('Date', axis=1)
    
    print "Created the time data features!"
    
    return data

In [5]:
sfpickle_file = '../../cs281_data/large_data/sfclean.pk'

In [ ]:
# We only need to do this once, afterwards we should load from the 
# saved location.
# DO NOT RUN IF PICKLED FILE ALREADY EXISTS
if not loadPickle:
    sfclean = cleanColumns(sfdata)
    sfclean = extractDateFeatures(sfclean)
    sfclean.to_pickle(sfpickle_file)

In [6]:
# Load sfclean from pickled location!
import pickle
sfclean = open(sfpickle_file)
sfclean = pickle.load(sfclean)

In [7]:
# Sort the data by time
sfclean = sfclean.sort_values(by='TimeFeature')

In [ ]:
# Save memory by deleting old data
# del(sfdata)
# del(sfclean_data)

In [ ]:
# Generate some additional histograms
columnsToPlotSF2 = { 'Month' : 'Month',
                    'Day of Month' : 'DoM',
                    'Year' : 'Year',
                   'Hour of Day' : 'ToD' }

In [ ]:
plotCrimeCounts(sfclean, columnsToPlotSF2, 'Boston')

In [ ]:
# Make histograms of the crimes by month for each given year
years = bosData.Year.unique()
for year in years:
    # Subset the data
    data = bosData[bosData.Year == year]
    
    # Make a histogram based on month
    sns.countplot(x = 'Month', data=data)
    plt.title('Boston Crime Histogram for {}'.format(year))
    plt.show()

In [7]:
# Repeat the above procedure but do it for the sf data
years = sfclean.Year.unique()
for year in years:
    data = sfclean[sfclean.Year == year]
    
    sns.countplot(x = 'Month', data=data)
    plt.title('San Francisco Crime Histogram for {}'.format(2001 + year))
    plt.show()

In [16]:
# Let's generate some heatmaps for both of these data crimes
# Can we overlay these on top of the geographical location???
n = 30
partitionedData = createPartitions(sfclean, n)

In [17]:
# Let's do one really large heatmap
def createHeatmapData(data):
    # data is what we're making a heatmap on.
    # Using the San Francisco data set, we create an nxn matrix
    # which counts the number of crimes in a given area!
    res = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            res[i,j] = len(data[(data.xRegion == i) & (data.yRegion == j)])
    return res

In [18]:
sns.heatmap(createHeatmapData(partitionedData))
plt.title('Crime Distribution in San Francisco')
plt.show()

In [21]:
# Let's redo the heatmap year by year?
years = partitionedData.Year.unique()
for year in years:
    data = partitionedData[partitionedData.Year == year]
    
    sns.heatmap(createHeatmapData(data))
    plt.title('Crime Distribution in San Francisco for {}'.format(year + 2001))
    plt.savefig('../figures/sf_data_analysis/heat_map_{}'.format(year + 2001))
    plt.close()

In [24]:
# Let's redo it month by month because that's what we want to analyze?
years = partitionedData.Year.unique()
months = partitionedData.Month.unique()
for year in years:
    for month in months:
        data = partitionedData[(partitionedData.Year == year) &
                               (partitionedData.Month == month)]
        sns.heatmap(createHeatmapData(data))
        plt.title('Crime Distribution in San Francisco for {}, {}'.format(month, year + 2001))
        plt.savefig('../figures/sf_data_analysis/heat_map_{}_{}'.format(month, year+2001))
        plt.close()

In [108]:
res = {}
steps = {}
def operationsToK(numbers, k, epsilon = 0.0001):
    try:
        return res[tuple(numbers),k]
    except KeyError:
        if len(numbers) == 1:
            return abs(numbers[0] - k) < epsilon
        for number in numbers:
            newSet = list(numbers)
            newSet.remove(number)
            # print newSet
            if operationsToK(newSet, k - number):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}-{}'.format(k,number)
                return True
            if operationsToK(newSet, number- k):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}-{}'.format(number,k)
                return True
            if operationsToK(newSet, number+ k):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}+{}'.format(k,number)
                return True
            if operationsToK(newSet, number * k):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}*{}'.format(k,number)
                return True
            if operationsToK(newSet, float(number) / k):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}/{}'.format(number,k)
                return True
            if operationsToK(newSet, k / float(number)):
                res[tuple(numbers), k] = True
                steps[tuple(numbers), k] = '{}/{}'.format(k,number)
                return True
        res[tuple(numbers), k] = False
        return False

In [110]:
res = {}
steps = {}
print operationsToK([1,5,5,5], 0.20000000000000018)

True


In [112]:
steps

{((1, 5, 5, 5), 0.20000000000000018): '0.2*1',
 ((5, 5), 1.0000000000000009): '1.0*5',
 ((5, 5, 5), 0.20000000000000018): '0.2*5'}

In [78]:
ls = [1,1]
ls.remove(1)

In [79]:
ls

[1]

In [80]:
tuple([1,1])

(1, 1)